In [1]:
import pickle
import pandas
from collections import Counter, defaultdict
import operator
import utils
from geopy.geocoders import Nominatim
from newspaper import Article
import requests
from urllib.parse import urlencode, quote_plus

In [2]:
urls_and_paths = [('frames/children_killed', 'http://www.gunviolencearchive.org/children-killed'),
                  ('frames/children_injured', 'http://www.gunviolencearchive.org/children-injured'),
                  ('frames/teens_killed', 'http://www.gunviolencearchive.org/teens-killed'),
                  ('frames/teens_injured', 'http://www.gunviolencearchive.org/teens-injured'),
                  ('frames/accidental_deaths', 'http://www.gunviolencearchive.org/accidental-deaths'),
                  ('frames/accidental_injuries', 'http://www.gunviolencearchive.org/accidental-injuries'),
                  ('frames/accidental_deaths_children', 'http://www.gunviolencearchive.org/accidental-child-deaths'),
                  ('frames/accidental_injuries_children', 'http://www.gunviolencearchive.org/accidental-child-injuries'),
                  ('frames/accidental_deaths_teens', 'http://www.gunviolencearchive.org/accidental-teen-deaths'),
                  ('frames/accidental_injuries_teens', 'http://www.gunviolencearchive.org/accidental-teen-injuries'),
                  ('frames/officer_involved_shootings', 'http://www.gunviolencearchive.org/officer-involved-shootings'),
                  ('frames/mass_shootings_2013', 'http://www.gunviolencearchive.org/reports/mass-shootings/2013'),
                  ('frames/mass_shootings_2014', 'http://www.gunviolencearchive.org/reports/mass-shootings/2014'),
                  ('frames/mass_shootings_2015', 'http://www.gunviolencearchive.org/reports/mass-shootings/2015'),
                  ('frames/mass_shootings', 'http://www.gunviolencearchive.org/mass-shooting')]
CORPUS_NAME = 'the_violent_corpus'

In [3]:
frames = []
for df_path, url in urls_and_paths:
    with open(df_path, 'rb') as infile:
        df = pickle.load(infile)
        frames.append(df)
df = pandas.concat(frames)
len(df)

5485

In [4]:
def get_sources(dataframe):
    """

    :param dataframe:
    :return:
    """
    sources = set()
    for index, row in dataframe.iterrows():
        sources.add(row['source_url'])
        sources.update(row['incident_sources'])
    return sources

In [5]:
sources=get_sources(df)

In [6]:
def generate_archive_uri(article_uri):
    archive_api='http://archive.org/wayback/available?'
    params={'url': article_uri}
    encoded_uri=archive_api + urlencode(params)
    print(encoded_uri)
    r=requests.get(encoded_uri)
    j=r.json()
    try:
        closest=j['archived_snapshots']['closest']
        if all([closest['available'], closest['status']=='200']):
            return closest['url']
    except:
        pass

archive_uri=generate_archive_uri('http://www.wdam.com/story/27682998/1-dead-4-injured-in-nightclub-shooting')

http://archive.org/wayback/available?url=http%3A%2F%2Fwww.wdam.com%2Fstory%2F27682998%2F1-dead-4-injured-in-nightclub-shooting


In [14]:
def download_and_parse_uri(url):
    language='en'
    a=Article(url, language)
    a.download()
    a.parse()
    print(a.title,'\nCONTENT:\n', a.text, '\nDATES1:\n', a.publish_date, '\nDATES2:\n', a.meta_data['date'])

download_and_parse_uri(archive_uri)

One dead, four injured in Waynesboro club shooting - WDAM-TV 7-News, Weather, Sports-Hattiesburg, MS 
CONTENT:
 One person is dead, and four others were injured in a shooting at a nightclub in Waynesboro.

Authorities told us 23-year-old Shaurice Cunningham died after being shot outside a club on Central Avenue.

Another person is reportedly in critical condition at a Jackson hospital.

The aunt of the deceased victim said she doesn't know why anyone would harm her nephew. “He was a good person. He was a father of two kids. He didn't bother nobody, whoever shot him, why and what reason,” Debra Cunningham said.

No arrests have been made, but police are working several leads and have several people of interest.

Nearby residents also expressed their concerns over the shooting.

“A young man lost his life last night; a few more young men were shot. No matter, no difference what they represented or what anyone says, they are still black people and we're supposed to come together in unity 